# 1. Imports

In [1]:
import s3fs
import pickle

import numpy as np
import regex as re
import pandas as pd
import umap.umap_ as umap

from sklearn import metrics
from datetime import datetime
from sqlalchemy import create_engine

from sklearn import cluster as c
from sklearn import preprocessing as pp

## 1.2. Load Dataset

In [2]:
path = '/home/matheusandrade/Documents/repos/clustering-loyalty-program-creation/'

df_raw = pd.read_csv(path + 'data/Ecommerce.csv', encoding='unicode_escape')

df_raw.drop(columns=['Unnamed: 8'], inplace=True)

# 2. Data Description

In [3]:
df1 = df_raw.copy()

## 2.1. Rename Columns

In [4]:
cols_new = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date', 'unit_price', 'customer_id', 'country']

df1.columns = cols_new

## 2.2. Data Dimension

In [5]:
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Cols: {}'.format(df1.shape[1]))

Number of Rows: 541909
Number of Cols: 8


## 2.3. Check NA

In [6]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 2.4. Replace NA

In [7]:
df_missing = df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]

In [8]:
# create reference
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000, 19000+len(df_backup), 1)

# merge original with reference dataframe
df1 = pd.merge(df1, df_backup, on='invoice_no', how='left')

# coalesce 
df1['customer_id'] = df1['customer_id_x'].combine_first( df1['customer_id_y'])

# drop extra columns
df1 = df1.drop(columns=['customer_id_x', 'customer_id_y'], axis=1)

In [9]:
df1.isna().sum()

invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
country            0
customer_id        0
dtype: int64

## 2.6. Check Data Types

In [10]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
country          object
customer_id     float64
dtype: object

## 2.4. Change Data Types

In [11]:
# invoice_date
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y')

# customer_id
df1['customer_id'] = df1['customer_id'].astype(int)

In [12]:
df1.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int64
dtype: object

## 2.7. Descriptive Statistics

In [13]:
num_attributes = df1.select_dtypes(include=['int64', 'int32', 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

### 2.7.1. Numerical Attributes

In [14]:
num_attributes.head()

,quantity,unit_price,customer_id
0,6,2.55,17850
1,6,3.39,17850
2,8,2.75,17850
3,6,3.39,17850
4,6,3.39,17850


In [15]:
# Central Tendency: mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# Dispersion: std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

# concatenate
m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,quantity,-80995.00,80995.0,161990.00,9.552250,3.00,218.080957,-0.264076,119769.160031
1,unit_price,-11062.06,38970.0,50032.06,4.611114,2.08,96.759764,186.506972,59005.719097
2,customer_id,12346.00,22709.0,10363.00,16688.840453,16249.00,2911.408666,0.487449,-0.804287


1. Quantity with negative value;

2. Unit price equal to zero.

### 2.7.2. Categorical Attributes

In [16]:
cat_attributes.head()

,invoice_no,stock_code,description,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
1,536365,71053,WHITE METAL LANTERN,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,United Kingdom


In [17]:
# invoice_no
df_letter_invoices = df1.loc[df1['invoice_no'].apply(lambda x: bool(re.search('[^0-9]+', x))), :]

print('Total number of invoices with letters: {}'.format(len(df_letter_invoices)))

print('Total number of negative quantities with letter in the invoice: {}'.format(len(df_letter_invoices[df_letter_invoices['quantity']< 0])))

Total number of invoices with letters: 9291
Total number of negative quantities with letter in the invoice: 9288


In [18]:
# stock_code
print('Number of stock codes with at least one letter: {}'.format(len(cat_attributes.loc[cat_attributes['stock_code'].apply(lambda x: bool(re.search('[^0-9]+', x))), 'stock_code'].drop_duplicates())))

print('List of stock codes composed only by letters: {}'.format(df1.loc[df1['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x))), 'stock_code'].unique()))


Number of stock codes with at least one letter: 1124
List of stock codes composed only by letters: ['POST' 'D' 'DOT' 'M' 'S' 'AMAZONFEE' 'm' 'DCGSSBOY' 'DCGSSGIRL' 'PADS'
 'B' 'CRUK']


In [19]:
# country
print('Number of countries in the database: {}'.format(len(df1['country'].unique())))
print('\nSome of these countries: \n{}'.format(df1['country'].value_counts(normalize=True).head(10)))

Number of countries in the database: 38

Some of these countries: 
United Kingdom    0.914320
Germany           0.017521
France            0.015790
EIRE              0.015124
Spain             0.004674
Netherlands       0.004375
Belgium           0.003818
Switzerland       0.003694
Portugal          0.002803
Australia         0.002323
Name: country, dtype: float64


# 3. Variable Filtering

In [20]:
df2 = df1.copy()

In [21]:
# unit price
df2 = df2.loc[df2['unit_price'] > 0.04, :]

# stock_code
df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'])]

# description
df2 = df2.drop(columns='description', axis=1)

# map
df2 = df2[~df2['country'].isin(['European Community', 'Unspecified'])]

# bad users
df2 = df2[~df2['customer_id'].isin( [16446] )]

# quantity
df2_returns = df2.loc[df2['quantity'] < 0, :]
df2_purchases = df2.loc[df2['quantity'] >= 0, :]

# 4. Feature Engineering

In [22]:
df3 = df2.copy()

## 4.1. Feature Creation

In [23]:
# data reference
df_ref = df3.drop(['invoice_no', 'stock_code', 'quantity', 'invoice_date', 'unit_price', 'country'], axis=1)
df_ref = df_ref.drop_duplicates(ignore_index=True)

### 4.1.1. Gross Revenue

In [24]:
# gross revenue (profit) quantity * price
df2_purchases.loc[:, 'gross_revenue'] = df2_purchases.loc[:, 'quantity'] * df2_purchases.loc[:, 'unit_price']

# monetary
df_monetary = df2_purchases[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')

/tmp/ipykernel_51050/1551767982.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_purchases.loc[:, 'gross_revenue'] = df2_purchases.loc[:, 'quantity'] * df2_purchases.loc[:, 'unit_price']


### 4.1.2. Recency

In [25]:
df_recency = df2_purchases.loc[:, ['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df2['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency.loc[:, ['customer_id', 'recency_days']].copy()
df_ref = pd.merge(df_ref, df_recency, on='customer_id', how='left')

### 4.1.5. Quantity of Products Purchased

In [26]:
df_freq = df2_purchases.loc[:, ['customer_id', 'stock_code']].groupby('customer_id').count().reset_index().rename(columns={'stock_code': 'q_products'})
df_ref = pd.merge(df_ref, df_freq, on='customer_id', how='left')

### 4.1.8. Frequency

In [27]:
df_aux = (df2_purchases[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                            .groupby('customer_id')
                                                            .agg(max_ = ( 'invoice_date', 'max'), 
                                                                 min_ = ( 'invoice_date', 'min'),
                                                                 days_= ( 'invoice_date', lambda x: ((x.max() - x.min()).days) + 1),
                                                                 buy_ = ( 'invoice_no', 'count'))).reset_index()

df_aux['frequency'] = df_aux[['buy_', 'days_']].apply(lambda x: x['buy_'] / x['days_'] if  x['days_'] != 0 else 0, axis=1)
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left')

### 4.1.9. Returns

In [28]:
df_returns = df2_returns[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity':'q_returns'})
df_returns['q_returns'] = df_returns['q_returns'] * -1

df_ref = pd.merge(df_ref, df_returns, how='left', on='customer_id')
df_ref.loc[df_ref['q_returns'].isna(), 'q_returns'] = 0


## 4.2. Check Dataframe

In [29]:
# check na
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
q_products       91
frequency        91
q_returns         0
dtype: int64

In [30]:
df_ref.head()

,customer_id,gross_revenue,recency_days,q_products,frequency,q_returns
0,17850,5391.21,372.0,297.0,17.000000,40.0
1,13047,3232.59,56.0,171.0,0.028302,35.0
2,12583,6705.38,2.0,232.0,0.040323,50.0
3,13748,948.25,95.0,28.0,0.017921,0.0
4,15100,876.00,333.0,3.0,0.073171,22.0


# 5. EDA (Exploratory Data Analysis)

In [31]:
cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'q_products', 'frequency', 'q_returns']
df_selected = df_ref[cols_selected].copy()

df4 = df_selected.dropna().copy()
df4.isna().sum()

customer_id      0
gross_revenue    0
recency_days     0
q_products       0
frequency        0
q_returns        0
dtype: int64

## 5.3. Space Study

In [32]:
# selected dataset
df4_3 = df4[cols_selected].drop(columns='customer_id', axis=1)

In [33]:
mm_gross_revenue = pp.MinMaxScaler()
mm_recency_days = pp.MinMaxScaler()
mm_q_products = pp.MinMaxScaler()
mm_frequency = pp.MinMaxScaler()
mm_q_returns = pp.MinMaxScaler()

df4_3['gross_revenue'] = mm_gross_revenue.fit_transform(df4_3[['gross_revenue']]) 
df4_3['recency_days'] = mm_recency_days.fit_transform(df4_3[['recency_days']]) 
df4_3['q_products'] = mm_q_products.fit_transform(df4_3[['q_products']]) 
df4_3['frequency'] = mm_frequency.fit_transform(df4_3[['frequency']]) 
df4_3['q_returns'] = mm_q_returns.fit_transform(df4_3[['q_returns']]) 

pickle.dump(mm_gross_revenue, open('../feature_transformation/mm_gross_revenue.pkl', 'wb'))
pickle.dump(mm_recency_days, open('../feature_transformation/mm_recency_days.pkl', 'wb'))
pickle.dump(mm_q_products, open('../feature_transformation/mm_q_products.pkl', 'wb'))
pickle.dump(mm_frequency, open('../feature_transformation/mm_frequency.pkl', 'wb'))
pickle.dump(mm_q_returns, open('../feature_transformation/mm_q_returns.pkl', 'wb'))

X = df4_3.copy()

### 5.3.2. UMAP

In [34]:
reducer = umap.UMAP(random_state=42)
pickle.dump(reducer, open('../feature_transformation/reducer.pkl', 'wb'))

embedding = reducer.fit_transform(X)

# embedding
df_umap = pd.DataFrame()
df_umap['embedding_x'] = embedding[:, 0]
df_umap['embedding_y'] = embedding[:, 1]

# 6. Data Preparation

In [35]:
df5 = df4_3.copy()
df5_aux = df4_3.copy()

# umap embedding
df5_umap = df_umap.copy()

# tsne embedding
#df5_tsne = df_tsne.copy()

# tree embedding
#df5_tree = df_tree.copy()

In [36]:
df5['gross_revenue'] = mm_gross_revenue.fit_transform(df5[['gross_revenue']])
df5['recency_days'] = mm_recency_days.fit_transform(df5[['recency_days']])
df5['q_products'] = mm_q_products.fit_transform(df5[['q_products']])
df5['frequency'] = mm_frequency.fit_transform(df5[['frequency']])
df5['q_returns'] = mm_q_returns.fit_transform(df5[['q_returns']])

# 7. Feature Selection

In [37]:
df6 = df5.copy()

df6_umap = df5_umap.copy()

#df6_tsne = df5_tsne.copy()

#df6_tree = df5_tree.copy()

# 8. Hyperparameter Fine Tuning

In [38]:
#X = df6.copy()

X_umap = df6_umap.copy()

#X_tsne = df6_tsne.copy()

#X_tree = df6_tree.copy()

# 9. Model Training

## 9.1. K-Means UMAP Embedding Space

In [39]:
# model definition
k = 11
kmeans_umap = c.KMeans(init='random', n_clusters=k, n_init=500, max_iter=300, random_state=42)

# model training
kmeans_umap.fit(X_umap)

# clustering
labels_umap_kmeans = kmeans_umap.predict(X_umap)

In [40]:
# WSS
print('WSS Value: {}'.format(kmeans_umap.inertia_))

# SS
print('SS Value: {}'.format(metrics.silhouette_score(X_umap, labels_umap_kmeans, metric='euclidean')))

WSS Value: 53153.93359375
SS Value: 0.5215659141540527


# 10. Cluster Analysis

## 10.1. KMeans UMAP Embedding Space

In [41]:
df9_umap_kmeans = X_umap
df9_umap_kmeans['cluster'] = labels_umap_kmeans

## 10.3. Clusters Profile

In [42]:
df9_2 = df4[cols_selected].copy()
df9_2['cluster'] = labels_umap_kmeans

# change data types
df9_2['cluster'] = df9_2['cluster'].astype('int64')
df9_2['recency_days'] = df9_2['recency_days'].astype('int64')
df9_2['q_products'] = df9_2['q_products'].astype('int64')
df9_2['q_returns'] = df9_2['q_returns'].astype('int64')

df9_2['last_training_timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

cluster_map = {4:0, 0:1, 6:2, 1:3, 3:4, 10:5, 7:6, 9:7, 5:8, 8:9, 2:10}
df9_2['cluster'] = df9_2['cluster'].map(cluster_map)

df9_2.head()

,customer_id,gross_revenue,recency_days,q_products,frequency,q_returns,cluster,last_training_timestamp
0,17850,5391.21,372,297,17.000000,40,5,2022-11-28 20:57:36
1,13047,3232.59,56,171,0.028302,35,2,2022-11-28 20:57:36
2,12583,6705.38,2,232,0.040323,50,1,2022-11-28 20:57:36
3,13748,948.25,95,28,0.017921,0,3,2022-11-28 20:57:36
4,15100,876.00,333,3,0.073171,22,4,2022-11-28 20:57:36


In [43]:
# number of customers
df_cluster = df9_2[['customer_id', 'cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_customer'] = 100 * (df_cluster['customer_id'] / df_cluster['customer_id'].sum())

# average gross revenue
df_avg_gross_revenue = df9_2[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_gross_revenue, how='inner', on='cluster')

# average recency days
df_avg_recency_days = df9_2[['recency_days', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_recency_days, how='inner', on='cluster')

# average invoice number
df_avg_invoice_no = df9_2[['q_products', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_invoice_no, how='inner', on='cluster')

# average frequency
df_avg_frequency = df9_2[['frequency', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_frequency, how='inner', on='cluster')

# average returns
df_avg_returns = df9_2[['q_returns', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_returns, how='inner', on='cluster')

In [44]:
df_cluster.sort_values('gross_revenue', ascending=False)

,cluster,customer_id,perc_customer,gross_revenue,recency_days,q_products,frequency,q_returns
0,0,755,13.257243,6260.089298,11.672848,241.884768,0.046377,76.582781
1,1,383,6.725198,2663.616136,4.229765,175.469974,0.140000,17.490862
2,2,836,14.679543,1705.616292,36.602871,98.068182,0.030811,16.688995
3,3,392,6.883231,1164.390612,100.165816,61.757653,0.187623,8.362245
4,4,429,7.532924,1028.458881,290.703963,59.675991,0.630727,202.198135
5,5,277,4.863916,906.624801,362.581227,65.126354,1.051473,2.472924
6,6,586,10.289728,861.548140,35.119454,44.726962,0.712406,3.452218
7,7,595,10.447761,774.428504,135.126050,65.057143,0.766010,3.692437
8,8,391,6.865672,647.621893,199.107417,47.230179,1.023018,2.447570
9,9,408,7.164179,606.151887,56.230392,46.142157,1.073063,6.595588


# 12. Deployment

In [45]:
df9_2.dtypes

customer_id                  int64
gross_revenue              float64
recency_days                 int64
q_products                   int64
frequency                  float64
q_returns                    int64
cluster                      int64
last_training_timestamp     object
dtype: object

## 12.1. Insert Into SQLite

In [46]:
# database connection
path_database = 'sqlite:///' + path + 'database/loyalty_program'
conn = create_engine(path_database)

# drop table
query_drop_table_loyalty_program = """
    DROP TABLE loyalty_program
"""

# create table
query_create_table_loyalty_program = """
    CREATE TABLE loyalty_program(
        customer_id INTEGER,
        gross_revenue REAL,
        recency_days INTEGER,
        q_products INTEGER,
        frequency REAL,
        q_returns INTEGER,
        cluster INTEGER
    )
"""

conn.execute(query_drop_table_loyalty_program)

# insert data
df9_2.to_sql('loyalty_program', con=conn, if_exists='append', index=False)

5695

In [47]:
query_all_data = """
    SELECT * FROM loyalty_program lp
"""

df = pd.read_sql_query(query_all_data, conn)

df.head()

,customer_id,gross_revenue,recency_days,q_products,frequency,q_returns,cluster,last_training_timestamp
0,17850,5391.21,372,297,17.000000,40,5,2022-11-28 20:57:36
1,13047,3232.59,56,171,0.028302,35,2,2022-11-28 20:57:36
2,12583,6705.38,2,232,0.040323,50,1,2022-11-28 20:57:36
3,13748,948.25,95,28,0.017921,0,3,2022-11-28 20:57:36
4,15100,876.00,333,3,0.073171,22,4,2022-11-28 20:57:36
